In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
tf.get_logger().setLevel('ERROR')


In [3]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

signal_real = pd.read_csv("./combined-dataset/real-signal-injected.csv", header=None).astype(float)
signal_syn = pd.read_csv("./combined-dataset/synthetic-signal-injected.csv", header=None).astype(float)
noise_real = pd.read_csv("./combined-dataset/real-noise.csv", header=None).astype(float)
noise_syn = pd.read_csv("./combined-dataset/synthetic-noise.csv", header=None).astype(float)

signal_real["label"] = 1
signal_syn["label"] = 1
noise_real["label"] = 0
noise_syn["label"] = 0

train_data = pd.concat([signal_real, signal_syn, noise_real, noise_syn], axis = 0)
train_data = shuffle(train_data, random_state = 42)

X = train_data.iloc[:, :-1].values
y = train_data["label"].values.astype(np.float32)

noise_all = pd.concat([noise_real, noise_syn], axis = 0)

scaler = StandardScaler()
scaler.fit(noise_all.iloc[:, :-1])
X = scaler.transform(X)

X = np.expand_dims(X, axis=-1)

In [4]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Dense, Dropout, LayerNormalization, Conv1D, MaxPooling1D,
    GlobalAveragePooling1D, Add, MultiHeadAttention, BatchNormalization
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# ========= ✅ 1. Preprocessing =========
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.squeeze(-1))  # -> (N, 16384)
X_scaled = X_scaled[..., np.newaxis]            # -> (N, 16384, 1)

X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

# ========= ✅ 2. Transformer Block =========
def transformer_block(x, head_size=32, num_heads=2, ff_dim=64, dropout=0.1):
    # Self-attention
    attn = MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(x, x)
    x = Add()([x, attn])
    x = LayerNormalization()(x)

    # Feed-forward
    ff = Dense(ff_dim, activation='relu')(x)
    ff = Dense(x.shape[-1])(ff)
    x = Add()([x, ff])
    x = LayerNormalization()(x)
    return x

# ========= ✅ 3. Model Builder =========
def build_cnn_transformer(input_shape=(16384, 1)):
    inp = Input(shape=input_shape)

    # --- CNN feature extractor ---
    x = Conv1D(32, 16, strides=4, padding='same')(inp)   # (4096, 32)
    x = BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = MaxPooling1D(pool_size=4)(x)                     # (1024, 32)

    x = Conv1D(64, 8, strides=2, padding='same')(x)      # (512, 64)
    x = BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = MaxPooling1D(pool_size=2)(x)                     # (256, 64)

    # --- Transformer block ---
    x = transformer_block(x, head_size=32, num_heads=2, ff_dim=64, dropout=0.1)

    # --- Classification ---
    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    out = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inp, outputs=out)
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
    return model

# ========= ✅ 4. Train =========
model = build_cnn_transformer()
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=8,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)],
    verbose=1
)

# ========= ✅ 5. Evaluate =========
y_pred = model.predict(X_val).ravel()
print("ROC AUC:", roc_auc_score(y_val, y_pred))
print(classification_report(y_val, (y_pred > 0.5).astype(int)))


Epoch 1/50
1792/1792 [==============================] - 29s 12ms/step - loss: 0.7064 - accuracy: 0.5041 - auc: 0.5068 - val_loss: 0.6951 - val_accuracy: 0.5000 - val_auc: 0.5243
Epoch 2/50
1792/1792 [==============================] - 20s 11ms/step - loss: 0.6974 - accuracy: 0.5043 - auc: 0.5081 - val_loss: 0.6932 - val_accuracy: 0.5028 - val_auc: 0.5723
Epoch 3/50
1792/1792 [==============================] - 19s 10ms/step - loss: 0.6959 - accuracy: 0.5061 - auc: 0.5084 - val_loss: 0.6943 - val_accuracy: 0.4933 - val_auc: 0.4977
Epoch 4/50
1792/1792 [==============================] - 20s 11ms/step - loss: 0.6964 - accuracy: 0.5016 - auc: 0.5028 - val_loss: 0.6932 - val_accuracy: 0.4997 - val_auc: 0.5098
Epoch 5/50
1792/1792 [==============================] - 20s 11ms/step - loss: 0.6952 - accuracy: 0.5012 - auc: 0.5019 - val_loss: 0.6930 - val_accuracy: 0.5717 - val_auc: 0.5048
Epoch 6/50
1792/1792 [==============================] - 21s 12ms/step - loss: 0.6943 - accuracy: 0.5010 - auc:

In [6]:
model.save("final_cnn_transformer_model.h5")